In [1]:
from gams.transfer import Container
import pandas as pd
import numpy as np
from utils import read_gdx,read_scenarios

In [2]:
cols = ['region_def','ssp','t','year','n','pop(mill)'] #names to be used for columns
data = pd.DataFrame(columns=cols)
region_defs = ['r5','witch17','ed57']

for region_def in region_defs:
    path = f'data_{region_def}/data_baseline.gdx'
    gdx = Container(path) #values for ssp_l (population), ssp_ykali (GDP_MER), ppp2mer
    reg_data = gdx['ssp_l'].records.rename(columns={'uni_0':'ssp','t_1':'t','n_2':'n','value':'pop(mill)'})
    reg_data['t'] = reg_data['t'].astype(int) #t as integer
    reg_data['year'] = 2010 + 5*reg_data['t'] #get year column
    reg_data['region_def'] = region_def #region definition (r5,witch17,ed57)
    reg_data = reg_data[cols] #reorganize columns
    data = pd.concat([data,reg_data])
data

,region_def,ssp,t,year,n,pop(mill)
0,r5,SSP1,1,2015,r5asia,3760.084
1,r5,SSP1,1,2015,r5lam,615.346
2,r5,SSP1,1,2015,r5maf,1314.522
3,r5,SSP1,1,2015,r5oecd,1037.214
4,r5,SSP1,1,2015,r5ref,483.682
...,...,...,...,...,...,...
16525,ed57,SSP5,9,2055,tur,86.588
16526,ed57,SSP5,9,2055,ukr,38.647
16527,ed57,SSP5,9,2055,usa,501.112
16528,ed57,SSP5,9,2055,vnm,94.396


In [3]:
ssps = [f'SSP{x}' for x in range(1,6)] #List of all ssps
shares = pd.DataFrame()

# For every region_def and ssp compute the avg population until 2100 and the corresponding pop_share
for region_def in region_defs:
    for ssp in ssps:
        new_data = data[(data['region_def'] == region_def) & (data['year'] <= 2100) & (data['ssp'] == ssp)] #  & (data['ssp'] == ssp)
        new_data = new_data.groupby(['n']).agg({'pop(mill)':['mean']}).reset_index()
        new_data['pop_share'] = new_data['pop(mill)'] / new_data['pop(mill)'].sum()
        new_data[['region_def','ssp']] = [region_def,ssp]
        shares = pd.concat([shares,new_data])

cols = ['region_def','ssp','n','avg_pop(mill)','pop_share']
shares.columns = shares.columns.droplevel(1)
shares = shares.rename(columns={'pop(mill)':'avg_pop(mill)'})[cols]
shares


,region_def,ssp,n,avg_pop(mill),pop_share
0,r5,SSP1,r5asia,3701.456444,0.467290
1,r5,SSP1,r5lam,622.102333,0.078537
2,r5,SSP1,r5maf,1977.885889,0.249698
3,r5,SSP1,r5oecd,1181.765722,0.149192
4,r5,SSP1,r5ref,437.894389,0.055282
...,...,...,...,...,...
52,ed57,SSP5,tur,80.489000,0.009935
53,ed57,SSP5,ukr,37.178444,0.004589
54,ed57,SSP5,usa,518.115667,0.063955
55,ed57,SSP5,vnm,86.768000,0.010711


In [5]:
for reg_def in region_defs:
    new_data = shares[shares['region_def'] == reg_def]
    new_data.to_csv(f'data_{reg_def}/pop_shares.csv',index=False)

shares.to_csv('region_pop_shares.csv',index=False)